# Azure AI Agent Service आणि Semantic Kernel वापरून RAG

हा कोड स्निपेट `Azure AI Agent Service` आणि `Semantic Kernel` वापरून retrieval-augmented generation (RAG) साठी Azure AI एजंट तयार करण्याची आणि व्यवस्थापित करण्याची प्रक्रिया दाखवतो. हा एजंट वापरकर्त्याच्या प्रश्नांवर आधारित संदर्भ शोधतो आणि त्यानुसार अचूक उत्तर प्रदान करतो.


SQLite आवृत्ती दुरुस्ती  
जर तुम्हाला खालील त्रुटी आली:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```  

तुमच्या नोटबुकच्या सुरुवातीला हा कोड ब्लॉक अनकॉमेंट करा:  


In [ ]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### पॅकेजेस आयात करणे
खालील कोड आवश्यक पॅकेजेस आयात करतो:


In [ ]:
# Azure imports for project client and credentials
from azure.ai.projects.models import FileSearchTool, OpenAIFile, VectorStore
from azure.identity.aio import DefaultAzureCredential

# Semantic Kernel imports
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread

# सेमॅंटिक कर्नल आणि Azure AI Agent Service सह Retrieval-Augmented Generation

हे उदाहरण दाखवते की **Azure AI Agent Service** वापरून **Retrieval-Augmented Generation (RAG)** कसे करायचे, ज्यामध्ये भाषा मॉडेलला अपलोड केलेल्या दस्तऐवजातील डोमेन-विशिष्ट संदर्भासह एकत्रित केले जाते.

### कसे कार्य करते

1. **दस्तऐवज अपलोड**: एक markdown फाइल (document.md) ज्यामध्ये माहिती (Contoso च्या प्रवास विमा धोरणाबद्दल) आहे, ती एजंट सेवेमध्ये अपलोड केली जाते.

2. **व्हेक्टर स्टोअर निर्मिती**: दस्तऐवजाचे अनुक्रमणिका तयार करून त्याच्या सामग्रीवर सेमॅंटिक शोध सक्षम केला जातो.

3. **एजंट कॉन्फिगरेशन**: `gpt-4o` मॉडेल वापरून एजंट तयार केला जातो, ज्यामध्ये खालील कठोर सूचना दिल्या जातात:
   - फक्त दस्तऐवजातून मिळालेल्या सामग्रीवर आधारित प्रश्नांची उत्तरे द्या.
   - जर प्रश्नाचा विषय मर्यादेबाहेर असेल तर उत्तर देण्यास नकार द्या.

4. **फाइल शोध साधन समाकलन**: `FileSearchTool` एजंटसोबत नोंदणीकृत केले जाते, ज्यामुळे मॉडेलला अनुमानादरम्यान अनुक्रमित दस्तऐवजातून संबंधित तुकडे शोधणे आणि पुनर्प्राप्त करणे शक्य होते.

5. **वापरकर्ता संवाद**: वापरकर्ते प्रश्न विचारू शकतात. जर दस्तऐवजामध्ये संबंधित माहिती सापडली, तर एजंट एक आधारभूत उत्तर तयार करतो.  
   जर नाही, तर एजंट स्पष्टपणे उत्तर देतो की दस्तऐवजामध्ये पुरेशी माहिती उपलब्ध नाही.


### मुख्य फंक्शन



सुनिश्चित करा की `DefaultAzureCredential` वापरताना योग्य प्रमाणीकरण संदर्भ प्रदान करण्यासाठी Azure CLI वापरून प्रथम `az login` चालवा. Azure AI Agent Service API कीज वापरत नाही.


In [ ]:
async def main():
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        file: OpenAIFile = await client.agents.upload_file_and_poll(file_path="document.md", purpose="assistants")
        vector_store: VectorStore = await client.agents.create_vector_store_and_poll(
            file_ids=[file.id], name="my_vectorstore"
        )

        # Define agent name and instructions tailored for RAG.
        AGENT_NAME = "RAGAgent"
        AGENT_INSTRUCTIONS = """
        You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

        - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
        "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
        - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
        - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
        - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
        - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

        You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
        """

        
        # Create file search tool with uploaded resources
        file_search = FileSearchTool(vector_store_ids=[vector_store.id])

        # 3. Create an agent on the Azure AI agent service with the file search tool
        agent_definition = await client.agents.create_agent(
            model="gpt-4o",  # This model should match your Azure OpenAI deployment.
            name=AGENT_NAME,
            instructions=AGENT_INSTRUCTIONS,
            tools=file_search.definitions,
            tool_resources=file_search.resources,
        )
        
        # Create the Azure AI Agent using the client and definition.
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )
        
        # Create a thread to hold the conversation
        # If no thread is provided, a new thread will be
        # created and returned with the initial response
        thread: AzureAIAgentThread | None = None
        
        # Example user queries.
        user_inputs = [
            "Can you explain Contoso's travel insurance coverage?",  # Relevant context.
            "What is a Neural Network?"  # No relevant context from the document. Will not contain a source annotation.
        ]
        
        try:
            for user_input in user_inputs:
                print(f"# User: '{user_input}'")
                # Invoke the agent for the specified thread for response
                async for response in agent.invoke(messages=user_input, thread=thread):
                    print(f"# {response.name}: {response}")
                    thread = response.thread
        finally:
            # Clean up resources.
            await thread.delete() if thread else None
            await client.agents.delete_vector_store(vector_store.id)
            await client.agents.delete_file(file.id)
            await client.agents.delete_agent(agent.id)
            print("\nCleaned up agent, thread, file, and vector store.")

await main()

ठीक आहे, कृपया तुमचा Markdown फाइल मजकूर द्या, मी त्याचा मराठीत अनुवाद करतो.



---

**अस्वीकरण**:  
हा दस्तऐवज AI भाषांतर सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) चा वापर करून भाषांतरित करण्यात आला आहे. आम्ही अचूकतेसाठी प्रयत्नशील असलो तरी, कृपया लक्षात घ्या की स्वयंचलित भाषांतरांमध्ये त्रुटी किंवा अचूकतेचा अभाव असू शकतो. मूळ भाषेतील दस्तऐवज हा अधिकृत स्रोत मानला जावा. महत्त्वाच्या माहितीसाठी व्यावसायिक मानवी भाषांतराची शिफारस केली जाते. या भाषांतराचा वापर केल्यामुळे उद्भवणाऱ्या कोणत्याही गैरसमज किंवा चुकीच्या अर्थासाठी आम्ही जबाबदार राहणार नाही.
